In [2]:
pip install mlflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached mlflow-2.12.1-py3-none-any.whl (20.2 MB)
  Using cached docker-7.0.0-py3-none-any.whl (147 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)
  Using cached alembic-1.13.1-py3-none-any.whl (233 kB)
  Using cached graphene-3.3-py2.py3-none-any.whl (128 kB)
  Using cached Mako-1.3.3-py3-none-any.whl (78 kB)
  Using cached graphql_relay-3.2.0-py3-none-any.whl (16 kB)
  Using cached graphql_core-3.2.3-py3-none-any.whl (202 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
import numpy as np 
import pandas as pd 
from imblearn.over_sampling import SMOTE
import mlflow
from mlflow.models import infer_signature

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")


# Exploring Dataframe's features

In [ ]:
leaky_data = pd.read_csv("predictive_maintenance.csv")
print(leaky_data["Type"].value_counts())
print('-----------------------------------------------')
print(leaky_data["Failure Type"].value_counts())
leaky_data.head()

L    6000
M    2997
H    1003
Name: Type, dtype: int64
-----------------------------------------------
No Failure                  9652
Heat Dissipation Failure     112
Power Failure                 95
Overstrain Failure            78
Tool Wear Failure             45
Random Failures               18
Name: Failure Type, dtype: int64


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [6]:
leaky_data.isna().sum()

UDI                        0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Target                     0
Failure Type               0
dtype: int64

In [7]:
leaky_data["Failure Type"].unique()

array(['No Failure', 'Power Failure', 'Tool Wear Failure',
       'Overstrain Failure', 'Random Failures',
       'Heat Dissipation Failure'], dtype=object)

In [8]:
NoFailure_data = leaky_data[leaky_data["Failure Type"] == "No Failure"]

print(NoFailure_data["Type"].value_counts())
NoFailure_data.head()

L    5757
M    2916
H     979
Name: Type, dtype: int64


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [9]:
Power_Failure = leaky_data[leaky_data["Failure Type"]=="Power Failure"]

print(Power_Failure["Type"].value_counts())
Power_Failure.head()

L    59
M    31
H     5
Name: Type, dtype: int64


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
50,51,L47230,L,298.9,309.1,2861,4.6,143,1,Power Failure
69,70,L47249,L,298.9,309.0,1410,65.7,191,1,Power Failure
168,169,L47348,L,298.4,308.3,1433,62.3,20,1,Power Failure
194,195,M15054,M,298.2,308.5,2678,10.7,86,1,Power Failure
207,208,M15067,M,298.4,308.7,1421,60.7,119,1,Power Failure


In [10]:
Heat_Dissipation_Failure = leaky_data[leaky_data["Failure Type"]=="Heat Dissipation Failure"]

print(Heat_Dissipation_Failure["Type"].value_counts())
Heat_Dissipation_Failure.head()

L    74
M    30
H     8
Name: Type, dtype: int64


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
3236,3237,M18096,M,300.8,309.4,1342,62.4,113,1,Heat Dissipation Failure
3760,3761,L50940,L,302.3,310.9,1377,46.8,166,1,Heat Dissipation Failure
3787,3788,L50967,L,302.3,310.8,1377,47.3,22,1,Heat Dissipation Failure
3793,3794,L50973,L,302.3,310.9,1379,49.3,34,1,Heat Dissipation Failure
3806,3807,M18666,M,302.3,310.9,1360,44.0,67,1,Heat Dissipation Failure


In [11]:
Random_Failures = leaky_data[leaky_data["Failure Type"]=="Random Failures"]

print(Random_Failures["Type"].value_counts())
Random_Failures.head()

L    12
H     4
M     2
Name: Type, dtype: int64


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
1221,1222,M16081,M,297.0,308.3,1399,46.4,132,0,Random Failures
1302,1303,L48482,L,298.6,309.8,1505,45.7,144,0,Random Failures
1748,1749,H31162,H,298.4,307.7,1626,31.1,166,0,Random Failures
2072,2073,L49252,L,299.6,309.5,1570,35.5,189,0,Random Failures
2559,2560,L49739,L,299.3,309.0,1447,50.4,140,0,Random Failures


In [12]:
Overstrain_Failure = leaky_data[leaky_data["Failure Type"]=="Overstrain Failure"]

print(Overstrain_Failure["Type"].value_counts())
Overstrain_Failure.head()

L    73
M     4
H     1
Name: Type, dtype: int64


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
160,161,L47340,L,298.4,308.2,1282,60.7,216,1,Overstrain Failure
161,162,L47341,L,298.3,308.1,1412,52.3,218,1,Overstrain Failure
242,243,L47422,L,298.0,308.2,1348,58.8,202,1,Overstrain Failure
248,249,L47428,L,298.0,308.3,1362,56.8,216,1,Overstrain Failure
249,250,L47429,L,298.0,308.3,1405,56.2,218,1,Overstrain Failure


In [13]:
Tool_Wear_Failure = leaky_data[leaky_data["Failure Type"]=="Tool Wear Failure"]

print(Tool_Wear_Failure["Type"].value_counts())
Tool_Wear_Failure.head()

L    25
M    14
H     6
Name: Type, dtype: int64


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
77,78,L47257,L,298.8,308.9,1455,41.3,208,1,Tool Wear Failure
1087,1088,H30501,H,296.9,307.8,1549,35.8,206,1,Tool Wear Failure
1509,1510,L48689,L,298.0,308.5,1429,37.7,220,1,Tool Wear Failure
1682,1683,H31096,H,297.9,307.4,1604,36.1,225,1,Tool Wear Failure
1763,1764,L48943,L,298.2,307.6,1511,31.0,209,1,Tool Wear Failure


In [14]:
grouped= leaky_data.copy()
grouped.drop(["Product ID","UDI","Target"],axis=1,inplace=True)
grouped = grouped.groupby(['Failure Type'])
statistics = grouped.describe()

statistics = grouped.describe().transpose()

statistics

Failure Type                   Heat Dissipation Failure   No Failure  \
Air temperature [K]     count                112.000000  9652.000000   
                        mean                 302.567857   299.972855   
                        std                    0.604371     1.990937   
                        min                  300.800000   295.300000   
                        25%                  302.100000   298.300000   
                        50%                  302.450000   300.000000   
                        75%                  303.125000   301.425000   
                        max                  303.700000   304.500000   
Process temperature [K] count                112.000000  9652.000000   
                        mean                 310.799107   309.994343   
                        std                    0.644938     1.486268   
                        min                  309.400000   305.700000   
                        25%                  310.300000   308.800000   
                        50%                  310.700000   310.000000   
                        75%                  311.300000   311.100000   
                        max                  312.200000   313.800000   
Rotational speed [rpm]  count                112.000000  9652.000000   
                        mean                1337.964286  1540.324389   
                        std                   34.371036   167.442595   
                        min                 1212.000000  1168.000000   
                        25%                 1323.250000  1429.000000   
                        50%                 1346.000000  1507.000000   
                        75%                 1363.250000  1615.000000   
                        max                 1379.000000  2695.000000   
Torque [Nm]             count                112.000000  9652.000000   
                        mean                  52.778571    39.624316   
                        std                    5.825285     9.468888   
                        min                   41.600000    12.600000   
                        25%                   48.200000    33.100000   
                        50%                   52.350000    39.900000   
                        75%                   56.350000    46.300000   
                        max                   67.800000    70.000000   
Tool wear [min]         count                112.000000  9652.000000   
                        mean                 107.339286   106.678927   
                        std                   64.427283    62.934391   
                        min                    2.000000     0.000000   
                        25%                   54.500000    52.000000   
                        50%                  106.000000   107.000000   
                        75%                  161.500000   160.000000   
                        max                  229.000000   246.000000   

Failure Type                   Overstrain Failure  Power Failure  \
Air temperature [K]     count           78.000000      95.000000   
                        mean           299.867949     300.075789   
                        std              1.954050       2.147127   
                        min            295.600000     295.700000   
                        25%            298.325000     298.400000   
                        50%            299.450000     300.400000   
                        75%            301.150000     301.800000   
                        max            304.000000     304.000000   
Process temperature [K] count           78.000000      95.000000   
                        mean           310.051282     309.954737   
                        std              1.552940       1.600450   
                        min            306.100000     306.200000   
                        25%            308.600000     308.700000   
                        50%            310.100000     310.200000   
                        75%

# In all failures except for random failures the "Target" column value is always one indicting failure , in conclusion this column is responsible for the data leakage

# Preprocessing 
Encoding & Handle Leakage &imbalance

In [15]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


# Label Encoding
label_encoder = LabelEncoder()
encoded_data = label_encoder.fit_transform(leaky_data["Failure Type"])

classes = label_encoder.classes_
encoded_values = label_encoder.transform(classes)

for i, class_name in enumerate(classes):
    print(f"{class_name}: {encoded_values[i]}")

leaky_encoded_df =leaky_data.copy()
leaky_encoded_df["Encoded Failure Type"] = encoded_data
leaky_encoded_df.drop("Failure Type",axis=1,inplace=True)
leaky_encoded_df.head()

Heat Dissipation Failure: 0
No Failure: 1
Overstrain Failure: 2
Power Failure: 3
Random Failures: 4
Tool Wear Failure: 5


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Encoded Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,1
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,1
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,1
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,1
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,1


In [16]:
onehot_encoder = OneHotEncoder(sparse=False, drop='first')
onehot_encoded = onehot_encoder.fit_transform(leaky_data[["Type"]])

# Creating DataFrame with one-hot encoded columns
onehot_df = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out(["Type"]))

# Concatenating original DataFrame with one-hot encoded DataFrame
leaky_encoded_df = pd.concat([leaky_encoded_df, onehot_df], axis=1)

leaky_encoded_df.drop(["Type"], axis=1, inplace=True)

leaky_encoded_df.head()


,UDI,Product ID,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Encoded Failure Type,Type_L,Type_M
0,1,M14860,298.1,308.6,1551,42.8,0,0,1,0.0,1.0
1,2,L47181,298.2,308.7,1408,46.3,3,0,1,1.0,0.0
2,3,L47182,298.1,308.5,1498,49.4,5,0,1,1.0,0.0
3,4,L47183,298.2,308.6,1433,39.5,7,0,1,1.0,0.0
4,5,L47184,298.2,308.7,1408,40.0,9,0,1,1.0,0.0


In [17]:
leaky_encoded_df["Product ID"].nunique()

10000

In [18]:
leaky_encoded_df.drop(["Product ID","UDI"],axis=1,inplace=True)

non_leaky_encoded_df= leaky_encoded_df.copy()

non_leaky_encoded_df.drop("Target",axis=1,inplace=True)

# 2. Model Training with and Without Leakage

In [21]:
from sklearn.model_selection import train_test_split

# non-leaky dataset
X_non_leaky = non_leaky_encoded_df.drop(columns=["Encoded Failure Type"])
y_non_leaky = non_leaky_encoded_df["Encoded Failure Type"]

smote = SMOTE()

# Apply SMOTE to generate synthetic samples
X_non_leaky, y_non_leaky = smote.fit_resample(X_non_leaky, y_non_leaky)

# Split non-leaky data
X_train_non_leaky, X_test_non_leaky, y_train_non_leaky, y_test_non_leaky = train_test_split(
    X_non_leaky, y_non_leaky, test_size=0.2, random_state=42)


# ---------------------------------------------------------------------------------------------

# leaky dataset
X_leaky = leaky_encoded_df.drop(columns=["Encoded Failure Type"])
y_leaky = leaky_encoded_df["Encoded Failure Type"]

X_leaky, y_leaky = smote.fit_resample(X_leaky, y_leaky)

# Split leaky data
X_train_leaky, X_test_leaky, y_train_leaky, y_test_leaky = train_test_split(
    X_leaky, y_leaky, test_size=0.2, random_state=42)



In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import warnings

# Suppress specific warning messages
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Train Logistic Regression model for leaky dataset
leaky_logistic_regression_model = LogisticRegression(max_iter=1000)  # Increase max_iter
leaky_logistic_regression_model.fit(X_train_leaky, y_train_leaky)

# Predictions using Leaky Logistic Regression model
leaky_logistic_regression_predictions = leaky_logistic_regression_model.predict(X_test_leaky)
leaky_logistic_regression_accuracy = accuracy_score(y_test_leaky, leaky_logistic_regression_predictions)
leaky_logistic_regression_report = classification_report(y_test_leaky, leaky_logistic_regression_predictions)

# Print evaluation metrics
print("Leaky Logistic Regression Model:")
print(f"Accuracy: {leaky_logistic_regression_accuracy}")
print("Classification Report:")
print(leaky_logistic_regression_report)

# ------------------------------------------------------------------------------------------------------------

# Train Logistic Regression model for non-leaky dataset
non_leaky_logistic_regression_model = LogisticRegression(max_iter=1000)  # Increase max_iter
non_leaky_logistic_regression_model.fit(X_train_non_leaky, y_train_non_leaky)

# Predictions using Non-leaky Logistic Regression model
non_leaky_logistic_regression_predictions = non_leaky_logistic_regression_model.predict(X_test_non_leaky)
non_leaky_logistic_regression_accuracy = accuracy_score(y_test_non_leaky, non_leaky_logistic_regression_predictions)
non_leaky_logistic_regression_report = classification_report(y_test_non_leaky, non_leaky_logistic_regression_predictions)

# Print evaluation metrics
print("-------------------------------------------------------")
print("Non-leaky Logistic Regression Model:")
print(f"Accuracy: {non_leaky_logistic_regression_accuracy}")
print("Classification Report:")
print(non_leaky_logistic_regression_report)


Leaky Logistic Regression Model:
Accuracy: 0.848312181645515
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93      1881
           1       0.68      0.57      0.62      1962
           2       0.91      0.94      0.93      1933
           3       0.98      0.99      0.98      1957
           4       0.68      0.68      0.68      1916
           5       0.89      0.97      0.92      1934

    accuracy                           0.85     11583
   macro avg       0.84      0.85      0.84     11583
weighted avg       0.84      0.85      0.84     11583

-------------------------------------------------------
Non-leaky Logistic Regression Model:
Accuracy: 0.8391608391608392
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.94      0.90      1881
           1       0.69      0.55      0.61      1962
           2       0.90      0.95      0.92      1933
           

In [35]:
from sklearn.naive_bayes import GaussianNB

# Train Naive Bayes model for leaky dataset
leaky_naive_bayes_model = GaussianNB()
leaky_naive_bayes_model.fit(X_train_leaky, y_train_leaky)

# Predictions using Leaky Naive Bayes model
leaky_naive_bayes_predictions = leaky_naive_bayes_model.predict(X_test_leaky)
leaky_naive_bayes_accuracy = accuracy_score(y_test_leaky, leaky_naive_bayes_predictions)
leaky_naive_bayes_report = classification_report(y_test_leaky, leaky_naive_bayes_predictions)

# Print evaluation metrics
print("Leaky Naive Bayes Model:")
print(f"Accuracy: {leaky_naive_bayes_accuracy}")
print("Classification Report:")
print(leaky_naive_bayes_report)

# ------------------------------------------------------------------------------------------------------------

# Train Naive Bayes model for non-leaky dataset
non_leaky_naive_bayes_model = GaussianNB()
non_leaky_naive_bayes_model.fit(X_train_non_leaky, y_train_non_leaky)

# Predictions using Non-leaky Naive Bayes model
non_leaky_naive_bayes_predictions = non_leaky_naive_bayes_model.predict(X_test_non_leaky)
non_leaky_naive_bayes_accuracy = accuracy_score(y_test_non_leaky, non_leaky_naive_bayes_predictions)
non_leaky_naive_bayes_report = classification_report(y_test_non_leaky, non_leaky_naive_bayes_predictions)

# Print evaluation metrics
print("-------------------------------------------------------")
print("Non-leaky Naive Bayes Model:")
print(f"Accuracy: {non_leaky_naive_bayes_accuracy}")
print("Classification Report:")
print(non_leaky_naive_bayes_report)


Leaky Naive Bayes Model:
Accuracy: 0.8279374946041612
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      1881
           1       0.79      0.51      0.62      1962
           2       0.82      0.93      0.87      1933
           3       0.93      0.82      0.87      1957
           4       0.63      0.86      0.73      1916
           5       0.96      0.93      0.95      1934

    accuracy                           0.83     11583
   macro avg       0.84      0.83      0.83     11583
weighted avg       0.84      0.83      0.82     11583

-------------------------------------------------------
Non-leaky Naive Bayes Model:
Accuracy: 0.7662090995424329
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.92      0.86      1881
           1       0.67      0.46      0.55      1962
           2       0.82      0.92      0.86      1933
           3       0.84   

In [19]:
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import warnings

# Suppress specific warning messages
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Train Logistic Regression model for leaky dataset
leaky_logistic_regression_model = LogisticRegression(**params) 
leaky_logistic_regression_model.fit(X_train_leaky, y_train_leaky)

# Predictions using Leaky Logistic Regression model
leaky_logistic_regression_predictions = leaky_logistic_regression_model.predict(X_test_leaky)
leaky_logistic_regression_accuracy = accuracy_score(y_test_leaky, leaky_logistic_regression_predictions)
leaky_logistic_regression_report = classification_report(y_test_leaky, leaky_logistic_regression_predictions)

# Print evaluation metrics
print("Leaky Logistic Regression Model:")
print(f"Accuracy: {leaky_logistic_regression_accuracy}")
print("Classification Report:")
print(leaky_logistic_regression_report)
leaky_accuracy = accuracy_score(y_test_leaky, leaky_logistic_regression_predictions)


# ------------------------------------------------------------------------------------------------------------

# Train Logistic Regression model for non-leaky dataset
non_leaky_logistic_regression_model = LogisticRegression(**params)  
non_leaky_logistic_regression_model.fit(X_train_non_leaky, y_train_non_leaky)

# Predictions using Non-leaky Logistic Regression model
non_leaky_logistic_regression_predictions = non_leaky_logistic_regression_model.predict(X_test_non_leaky)
non_leaky_logistic_regression_accuracy = accuracy_score(y_test_non_leaky, non_leaky_logistic_regression_predictions)
non_leaky_logistic_regression_report = classification_report(y_test_non_leaky, non_leaky_logistic_regression_predictions)

# Print evaluation metrics
print("-------------------------------------------------------")
print("Non-leaky Logistic Regression Model:")
print(f"Accuracy: {non_leaky_logistic_regression_accuracy}")
print("Classification Report:")
print(non_leaky_logistic_regression_report)
nonleaky_accuracy = accuracy_score(y_test_non_leaky, non_leaky_logistic_regression_predictions)


Leaky Logistic Regression Model:
Accuracy: 0.8243114909781576
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.93      0.89      1881
           1       0.64      0.57      0.60      1962
           2       0.89      0.94      0.91      1933
           3       0.98      0.98      0.98      1957
           4       0.67      0.57      0.62      1916
           5       0.86      0.96      0.91      1934

    accuracy                           0.82     11583
   macro avg       0.82      0.82      0.82     11583
weighted avg       0.82      0.82      0.82     11583

-------------------------------------------------------
Non-leaky Logistic Regression Model:
Accuracy: 0.8259518259518259
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      1881
           1       0.66      0.51      0.58      1962
           2       0.90      0.93      0.91      1933
          

In [29]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("nonleaky_accuracy", nonleaky_accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for non leaky data")

    # Infer the model signature
    signature = infer_signature(X_train_non_leaky, non_leaky_logistic_regression_model.predict(X_train_non_leaky))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=non_leaky_logistic_regression_model,
        artifact_path="non_leaky_logistic_regression_model",
        signature=signature,
        input_example=X_train_non_leaky,
        registered_model_name="tracking-quickstart",
    )


Successfully registered model 'tracking-quickstart'.
2024/04/30 03:15:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1
Created version '1' of model 'tracking-quickstart'.


In [32]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test_non_leaky)

iris_feature_names = non_leaky_encoded_df.columns

result = pd.DataFrame(X_test_non_leaky, columns=iris_feature_names)
result["actual_class"] = y_test_non_leaky
result["predicted_class"] = predictions

result[:4]


2024/04/30 03:17:41 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Encoded Failure Type,Type_L,Type_M,actual_class,predicted_class
16971,302.178691,310.414764,1378,47.649162,166,NaN,0.393455,0.606545,0,0
53134,299.333911,309.670620,1517,34.894738,211,NaN,1.000000,0.000000,5,5
46497,298.978181,309.686546,1529,41.842558,161,NaN,1.000000,0.000000,4,4
9584,299.300000,310.500000,1529,41.300000,17,NaN,1.000000,0.000000,1,1
